In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# データセットの前処理を定義
transform = transforms.Compose([
    transforms.ToTensor(),  # PIL Imageまたはnumpy.ndarrayをPyTorchのTensorに変換
    transforms.Normalize((0.5,), (0.5,))  # ピクセル値の正規化
])

# MNISTデータセットのダウンロードとロード
trainset = MNIST(root='./data', train=True, download=True, transform=transform)
testset = MNIST(root='./data', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# ニューラルネットワークの定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28)  # フラット化
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# モデル、損失関数、最適化手法の設定
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# モデルのトレーニング
def train(model, trainloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for images, labels in trainloader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# テストデータでモデルの評価
def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

# トレーニングとテストの実行
train(model, trainloader, criterion, optimizer)
test(model, testloader)


Epoch 1, Loss: 0.09347999840974808
Epoch 2, Loss: 0.1326158195734024
Epoch 3, Loss: 0.03420419991016388
Epoch 4, Loss: 0.07114642858505249
Epoch 5, Loss: 0.06630312651395798
Accuracy: 97.35%


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# データセットの前処理を定義
transform = transforms.Compose([
    transforms.ToTensor(),  # PIL Imageまたはnumpy.ndarrayをPyTorchのTensorに変換
    transforms.Normalize((0.5,), (0.5,))  # ピクセル値の正規化
])

# MNISTデータセットのダウンロードとロード
trainset = MNIST(root='./data', train=True, download=True, transform=transform)
testset = MNIST(root='./data', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# ニューラルネットワークの定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# GPUの確認と使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} device')

# モデル、損失関数、最適化手法の設定
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# モデルのトレーニング
def train(model, trainloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# テストデータでモデルの評価
def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

# トレーニングとテストの実行
train(model, trainloader, criterion, optimizer)
test(model, testloader)

Using cuda device
Epoch 1, Loss: 0.238291934132576
Epoch 2, Loss: 0.06327597796916962
Epoch 3, Loss: 0.16367687284946442
Epoch 4, Loss: 0.1712600737810135
Epoch 5, Loss: 0.0334545373916626
Accuracy: 96.78%
